In [11]:
import urllib
import xml.etree.ElementTree as ET
import urllib
import re
import io, random
import pandas as pd
import time
from datetime import datetime
from sklearn.svm import SVR
import datetime
from skmultiflow.meta import AdaptiveRandomForestRegressor

In [12]:
df=pd.read_csv("merged_final.csv")

In [13]:
df.head()

,Unnamed: 0,DATE_TIME,PLANT_ID,SCE_KEY,DC_POWER,AC_POWER,DAILY_YIELD,TOTAL_YIELD,PLANT_ID.1,SOURCE_KEY,AMBIENT_TEMPERATURE,MODULE_TEMPERATURE,IRRADIATION
0,0,15-05-2020 00:00,4135001,1BY6WEcLGh8j5v7,0.0,0.0,0.0,6259559.0,4135001,HmiyD2TTLFNqkNe,25.184316,22.857507,0.0
1,1,15-05-2020 00:00,4135001,1IF53ai7Xc0U56Y,0.0,0.0,0.0,6183645.0,4135001,HmiyD2TTLFNqkNe,25.184316,22.857507,0.0
2,2,15-05-2020 00:00,4135001,3PZuoBAID5Wc2HD,0.0,0.0,0.0,6987759.0,4135001,HmiyD2TTLFNqkNe,25.184316,22.857507,0.0
3,3,15-05-2020 00:00,4135001,7JYdWkrLSPkdwr4,0.0,0.0,0.0,7602960.0,4135001,HmiyD2TTLFNqkNe,25.184316,22.857507,0.0
4,4,15-05-2020 00:00,4135001,McdE0feGgRqW7Ca,0.0,0.0,0.0,7158964.0,4135001,HmiyD2TTLFNqkNe,25.184316,22.857507,0.0


In [14]:
def pred_svr(x, y, x_test):
    svr_rbf = SVR(kernel='rbf', C=1e3, gamma=0.1)
    y_rbf = svr_rbf.fit(x, np.ravel(y,order='C'))
    return svr_rbf.predict(x_test)

def pred_hoe(x, y, x_test):
    arf_reg = AdaptiveRandomForestRegressor(random_state=123456)
    arf_reg.fit(np.array(x), np.ravel(y,order='C'))
    return arf_reg.predict(x_test)


def algorithms_pred(df):
    x_total = df[["AMBIENT_TEMPERATURE","MODULE_TEMPERATURE","IRRADIATION"]].to_numpy().tolist()
    y_total = df[["DAILY_YIELD"]].to_numpy().tolist()
    x = x_total[:len(df)-1]
    y = y_total[:len(df)-1]
    x_test = x_total[len(df)-1:]
    y_test = y_total[len(df)-1:]
    y_pred_svr = pred_svr(x,y,x_test)
    y_pred_hoe = pred_hoe(x,y,x_test)
    y_pred_ensemb = np.add(y_pred_hoe,y_pred_svr)
    if(y_test!=0):
        MAPE_list.append([y_test[0][0],y_pred_svr[0]])
        MAPE_list2.append([y_test[0][0],y_pred_hoe[0]])
        MAPE_list_ensemb.append([y_test[0][0],y_pred_ensemb[0]])
        


# def AMWR(df):
#     x_total = df[["AMBIENT_TEMPERATURE","MODULE_TEMPERATURE","IRRADIATION"]].to_numpy().tolist()
#     y_total = df[["DAILY_YIELD"]].to_numpy().tolist()
#     x = x_total[:len(df)-1]
#     y = y_total[:len(df)-1]
#     x_test = x_total[len(df)-1:]
#     y_test = y_total[len(df)-1:]
#     y_pred = pred(x,y,x_test)
#     if(y_test!=0):
#         MAPE_list.append([y_test[0][0],y_pred[0]])

In [15]:
def dataprocess(df,dt_start):
    dt_end=dt_start+datetime.timedelta(0,TrainingWindow*time_sampling)
    #print(dt_start,dt_end)
    #print(sensor_list)
    #df = df.set_index(['DATE_TIME'])
    #print(df.loc[dt_start:dt_end])
    df['DATE_TIME'] = pd.to_datetime(df['DATE_TIME'])
    mask = (df['DATE_TIME'] > dt_start) & (df['DATE_TIME'] <= dt_end)
    df_proc=df.loc[mask]
    for i in sensor_list:
        msk=(df_proc['SCE_KEY']==i[0])
        df_proc_sen=df_proc.loc[msk]
        algorithms_pred(df_proc_sen)


# def dataprocess(df,dt_start):
#     dt_end=dt_start+datetime.timedelta(0,TrainingWindow*time_sampling)
#     #print(dt_start,dt_end)
#     #print(sensor_list)
#     #df = df.set_index(['DATE_TIME'])
#     #print(df.loc[dt_start:dt_end])
#     df['DATE_TIME'] = pd.to_datetime(df['DATE_TIME'])
#     mask = (df['DATE_TIME'] > dt_start) & (df['DATE_TIME'] <= dt_end)
#     df_proc=df.loc[mask]
#     for i in sensor_list:
#         msk=(df_proc['SCE_KEY']==i[0])
#         df_proc_sen=df_proc.loc[msk]
#         AMWR(df_proc_sen)
    
    

In [20]:
#constant parameters
TrainingWindow = 25    #window size is found from historical data
PredictionWindow = 3   #prediction window/prediction horizon
time_sampling = 900 #Should be equal to data refreshing time
sensor_list=df[["SCE_KEY"]].to_numpy().tolist()
sensor_list=sensor_list[0:20]


if __name__ == '__main__':
    min_MAPE_svr = 100
    tw_min_MAPE_svr = 0
    min_MAPE_hoeff= 100
    tw_min_MAPE_hoeff = 0
    min_MAPE_ensemb= 100
    tw_min_MAPE_ensemb = 0
    for i in range(1,30):
        TrainingWindow+=1
        MAPE_list = list()
        MAPE_list2 = list()
        MAPE_list_ensemb = list()
        date=datetime.datetime(2020, 5, 15, 0, 0)
        end_date = datetime.datetime(2020, 5, 25, 6, 0)
        dataprocess(df,date)
        MAPE_npArray = np.array(MAPE_list)
        MAPE_npArray2 = np.array(MAPE_list2)
        MAPE_npArray_ensemb = np.array(MAPE_list_ensemb)
        #print(MAPE_npArray[:,0])
        mape = np.mean(np.abs((MAPE_npArray[:,0] - MAPE_npArray[:,1])/MAPE_npArray[:,0]))*100
        mape2 = np.mean(np.abs((MAPE_npArray2[:,0] - MAPE_npArray2[:,1])/MAPE_npArray2[:,0]))*100
        mape_ensemb = np.mean(np.abs((MAPE_npArray_ensemb[:,0] - MAPE_npArray_ensemb[:,1])/MAPE_npArray_ensemb[:,0]))*100
        if(min_MAPE_svr>mape):
            min_MAPE_svr = min(min_MAPE_svr, mape)
            tw_min_MAPE_svr = TrainingWindow
        if(min_MAPE_hoeff>mape2):
            min_MAPE_hoeff = min(min_MAPE_hoeff, mape2)
            tw_min_MAPE_hoeff = TrainingWindow
        if(min_MAPE_svr>mape_ensemb):
            min_MAPE_ensemb = min(min_MAPE_ensemb,mape_ensemb)
            tw_min_MAPE_ensemb = TrainingWindow
        print("MAPE svr is: "+ str(mape)+"Training window is: "+str(TrainingWindow) )
        print("MAPE hoeff is: "+ str(mape2)+"Training window is: "+str(TrainingWindow) )
        print("MAPE ensemb is: "+ str(mape_ensemb)+"Training window is: "+str(TrainingWindow) )
    print("min mape for svr is: "+str(min_MAPE_svr)+ " for trainingwindow: "+str(tw_min_MAPE_svr))
    print("min mape for hoeff: "+str(min_MAPE_hoeff)+ " for trainingwindow: "+str(tw_min_MAPE_hoeff))
    print("min mape for ensemb is: "+str(min_MAPE_ensemb)+ " for trainingwindow: "+str(tw_min_MAPE_ensemb))

#main function
# if __name__ == '__main__':
#     for i in range(1,90):
#         TrainingWindow+=1
#         MAPE_list = list()
#         date=datetime.datetime(2020, 5, 15, 0, 0)
#         end_date = datetime.datetime(2020, 5, 25, 6, 0)
#         dataprocess(df,date)
#         MAPE_npArray = np.array(MAPE_list)
#         mape = np.mean(np.abs((MAPE_npArray[:,0] - MAPE_npArray[:,1])/MAPE_npArray[:,0]))*100
#         print("MAPE is: "+ str(mape)+"Training window is: "+str(TrainingWindow) )

MAPE svr is: 97.24998290951781Training window is: 26
MAPE hoeff is: 74.452593794348Training window is: 26
MAPE ensemb is: 71.70257670386579Training window is: 26
MAPE svr is: 97.36525311309062Training window is: 27
MAPE hoeff is: 56.16220369385195Training window is: 27
MAPE ensemb is: 53.52745680694255Training window is: 27
MAPE svr is: 66.18640768630503Training window is: 28
MAPE hoeff is: 53.006129630389275Training window is: 28
MAPE ensemb is: 19.19253731669432Training window is: 28
MAPE svr is: 32.930743088003574Training window is: 29
MAPE hoeff is: 47.17186640983872Training window is: 29
MAPE ensemb is: 19.89739050215771Training window is: 29
MAPE svr is: 25.285583857562266Training window is: 30
MAPE hoeff is: 45.35970404723585Training window is: 30
MAPE ensemb is: 29.354712095201872Training window is: 30
MAPE svr is: 22.309054399891206Training window is: 31
MAPE hoeff is: 45.363842710836735Training window is: 31
MAPE ensemb is: 32.327102889272055Training window is: 31
MAPE svr is

In [ ]:
def pred_svr(x, y, x_test):
    svr_rbf = SVR(kernel='rbf', C=1e3, gamma=0.1)
    y_rbf = svr_rbf.fit(x, np.ravel(y,order='C'))
    return svr_rbf.predict(x_test)

def pred_randomforest(x, y, x_test):
    arf_reg = AdaptiveRandomForestRegressor(random_state=123456)
    arf_reg.fit(np.array(x), np.ravel(y,order='C'))
    return arf_reg.predict(x_test)

def pred_hoe(x, y, x_test):
    arf_reg = HoeffdingTreeRegressor()
    arf_reg.fit(np.array(x), np.ravel(y,order='C'))
    return arf_reg.predict(x_test)

def algorithms_pred(df):
    x_total = df[["AMBIENT_TEMPERATURE","MODULE_TEMPERATURE","IRRADIATION"]].to_numpy().tolist()
    y_total = df[["DAILY_YIELD"]].to_numpy().tolist()
    x = x_total[:len(df)-1]
    y = y_total[:len(df)-1]
    x_test = x_total[len(df)-1:]
    y_test = y_total[len(df)-1:]
    y_pred_svr = pred_svr(x,y,x_test)
    y_pred_hoe = pred_hoe(x,y,x_test)
    y_pred_rff = pred_randomforest(x,y,x_test)
    y_pred_ensemb = (1/3)*np.add(y_pred_hoe,y_pred_svr,y_pred_rff)
    
    if(y_test!=0):
        MAPE_list.append([y_test[0][0],y_pred_svr[0]])
        MAPE_list2.append([y_test[0][0],y_pred_hoe[0]])
        MAPE_list3.append([y_test[0][0],y_pred_rff[0]])
        MAPE_list_ensemb.append([y_test[0][0],y_pred_ensemb[0]])